<a href="https://www.kaggle.com/code/monicacruzsilva/pima-indians-diabetes-notebook?scriptVersionId=181504133" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Analytics libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Load Dataset

In [ ]:
data = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')

# Exploratory Data Analysis (EAD)

In [ ]:
# # Visualizando as primeiras linhas do DataFrame
print("Primeiras linhas do DataFrame:")

data.head()

In [ ]:
data.info()

In [ ]:
# Summary statistics - numeric only
print("\nSummary statistics of the dataset:")
print(data.describe())

* BloodPressure, Glucose, SkinThickness,Insulin, BMI  = 0 can be a outlier

In [ ]:
# total de observações
data.shape[0]

In [ ]:
# Check for missing values and data types
datadict = pd.DataFrame(data.dtypes)
datadict.columns = ["Dtype"]
datadict['Valores_Nulos'] = data.isnull().sum()
datadict['%_Valores_Nulos'] = (data.isnull().sum() / data.shape[0]) * 100
datadict['NUnique']=data.nunique()
datadict

In [ ]:
### seleciona variaveis numericas
num_data = data.select_dtypes(include=['float64', 'int64'])

## calcula a matriz de correlação
corr = num_data.corr()

# Plot
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='viridis', fmt=".2f", linewidths=0.5)
plt.title('Matriz de Correlacao')
plt.show()

In [ ]:
# Histograma das variáveis numéricas
data.hist(figsize=(12, 10))
plt.suptitle('Histogramas das Variáveis Numéricas')
plt.show()

In [ ]:
# Boxplot para variáveis numéricas em relação à variável alvo
plt.figure(figsize=(12, 8))

# Selecionando as variáveis numéricas
numeric_columns = data.select_dtypes(include=['int', 'float'])

for i, coluna in enumerate(numeric_columns):
    plt.subplot(3, 3, i + 1)
    sns.boxplot(x=data['Outcome'], y=data[coluna])
    plt.title(coluna)

plt.tight_layout()
plt.show()

# **Fitting Model**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Separando os dados em features (X) e target (y)
X = data.drop('Outcome', axis=1)
y = data['Outcome']

# Dividindo os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [ ]:
# Inicializando e ajustando o modelo de regressão logística
modelo = LogisticRegression(solver='lbfgs', max_iter=200)

In [ ]:
modelo.fit(X_train, y_train)

In [ ]:
# Fazendo previsões no conjunto de teste
y_predict = modelo.predict(X_test)

# Accuracy Score

In [ ]:
from sklearn.metrics import accuracy_score

# Calculando a precisão do modelo
precisao = accuracy_score(y_test, previsoes)
print("Precisão do modelo de regressão logística:", precisao)

# **Confusion Matrix**

In [ ]:
from sklearn.metrics import confusion_matrix

# Calculando a matriz de confusão
matriz_confusao = confusion_matrix(y_test, previsoes)

# Plotando a matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(matriz_confusao, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel('Previsão')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

# **Metrics**

In [ ]:
from sklearn.metrics import classification_report

# Avaliando o modelo
relatorio_classificacao = classification_report(y_test, previsoes)
print("Relatório de Classificação:\n", relatorio_classificacao)